## Hypothesis Test Example

This notebook performs an hypothesis test and computes the significance of the test (p-value) using the asympotic approximation of the profile likelihood function.

The test statistics for computing the hypothesis test is 

$$q_\mu = - 2 \log \frac { L( x \ | \ \mu , \hat{\hat{\nu} } ) }{  L( x \ | \ \hat{\mu} , \hat{\nu}  ) } \  \  \ \mathrm{for} \  \  
\hat{\mu}  >  0$$
$$q_\mu = 0     \hspace{3cm}     \ \mathrm{for} \  \  
\hat{\mu} \le  0$$

The p-value (significance) of the test is computed assuming that $q_\mu$ is distributed according to a 1/2 $\chi^2$ distribution with 1 degree of freedom.



In [ ]:
//%jsroot on

In [ ]:
using namespace RooStats;

In [ ]:
RooPlot * plot = nullptr;
RooAbsPdf * pdf = nullptr;
HypoTestResult * result = nullptr;
// enable use of NLL offset for better minimizations
RooStats::UseNLLOffset(true);

####  1. Reading the model (Workspace) from input file

First part is just to access the workspace file and retrieve the model and the data 

In [ ]:
TString fileName ="HiggsBinModel.root";  
//TString fileName ="CountingModel.root";  
TString workspaceName = "w";
TString modelConfigName = "ModelConfig";
TString dataName = "data";
TString integrationType = "";  
//ROOT::Math::MinimizerOptions::SetDefaultMinimizer("Minuit2");

nexp_events = 250;  // number of expected events for the expected significance

In [ ]:
auto file = TFile::Open(fileName);

In [ ]:
auto w =  (RooWorkspace*) file->Get(workspaceName);
w->Print();
auto sbModel = (RooStats::ModelConfig*) w->obj(modelConfigName);
auto  data = w->data(dataName);
auto poi = (RooRealVar*) sbModel->GetParametersOfInterest()->first();

##### Make the b Model by cloning the b model and use a value = 0 for the parameter of interest

In [ ]:
auto bModel = (RooStats::ModelConfig*) sbModel->Clone();
sbModel->SetName("S+B Model");
// save previous poi value
poiHat = poi->getVal(); 
poi->setVal(nexp_events);
sbModel->SetSnapshot( *poi);
bModel->SetName("B Model");
poi->setVal(0);
bModel->SetSnapshot( *poi  );
sbModel->Print();
bModel->Print();
// restore poi value to best fit value
poi->setVal(poiHat);

#### 2. Plot of the model pdf and the data 

we plot the pdf describing the model over the data. Since we have performed before a best fit, the pdf should have the parameters at their best fit values. 

In [ ]:
plot = w->var("x")->frame();
pdf = sbModel->GetPdf();
data->plotOn(plot);
pdf->plotOn(plot);
pdf->plotOn(plot, RooFit::Components("bmodel"),RooFit::LineStyle(kDashed));
pdf->plotOn(plot, RooFit::Components("smodel"),RooFit::LineColor(kRed));
pdf->paramOn(plot);

plot->Draw();
gPad->Draw();

We assume we know the mass and the width of the peak, so we set it to constant.

We could eventually also set to constant the background parameters. In this case the systematic uncertainties due to the background shape will not be taken into account.

In [ ]:
//w->var("a1")->setConstant(true);
//w->var("a2")->setConstant(true);
w->var("mass")->setConstant(true);
w->var("width")->setConstant(true);

In [ ]:
// RooStats::AsymptoticCalculator::SetPrintLevel(-1);  // to switch off print level 
RooStats::AsymptoticCalculator  asymCalc(*data, *sbModel, *bModel);

Configure the calculator

In [ ]:
asymCalc.SetOneSidedDiscovery(true);  // for one-side discovery test
//asymCalc.SetPrintLevel(-1);  // to suppress print level 

Run the calculator and get the result

In [ ]:
result = asymCalc.GetHypoTest();
result->Print();

In [ ]:
std::cout << "Significance = " << result->Significance() << " for p-value = " << result->NullPValue() << std::endl; 

#### Is Test statistic distribution like a chi-square distribution with n.d.f =1 ? 

We want to fit the null test statistic distribution to check if it is compatible with a chi2 distribution

Create the fit function as a 1/2 chisquared. Special case for forst bin (x < 0.05) 
Also the quantity plotted is the log-likelihood ratio and not 2 x log-likelihood ratio .
0.05 is the histogram bin width. 

In [ ]:
auto fchi2 = new TF1("chi2","[](double*x,double*p){ if (x[0] < 0.05) { return 0.5*p[0]+ 0.5*p[0]*ROOT::Math::chisquared_cdf(0.1,p[1]); } else { return 0.05*p[0]*ROOT::Math::chisquared_pdf(x[0],p[1]); } }",0.,20.,2,1);

In [ ]:
fchi2->SetParameters(1,1);
fchi2->SetNpx(1000);
fchi2->SetLineColor(kRed);
fchi2->Draw();
fchi2->SetTitle("Half-chi2 distribution");
gPad->SetLogy(true);
gPad->Draw();

In [ ]:
// add data value
sqmu = ROOT::Math::normal_quantile_c(result->NullPValue(),1);
qmu = sqmu*sqmu; 
std::cout << "test statistic data value = " << qmu << std::endl;
dataLine = new TLine(qmu,fchi2->GetMinimum(),qmu,1.);
dataLine->SetLineWidth(2);
dataLine->Draw();
gPad->Draw();